# Bouc-Wen Case Study

The purpose of this case study is to compare various model calibration methods on a simulated system displaying either simple linear or complex nonlinear behavior. We select a single-degree-of-freedom oscillator with a Bouc-Wen switch component BW-SDOF as our representative system. The basis for this example is derived from a case study by Wu & Smyth<sup>1</sup>, where the authors compared the performance of the unscented and extended Kalman filters on a single-degree-of-freedom Bouc-Wen system. We use the same parameters as given in this example and add a switch component between the linear and nonlinear displacement restoring forces of the system. The system model is visualized below.

<img src="03-Figures/BW-SDOF_system.png" width="50%">

### Deterministic Equations of Motion

The dynamics of the Bouc-Wen system are given by 

$$
m \ddot{x}_{BW}(t) + c \dot{x}_{BW}(t) + \left(1 - \alpha \right) kx_{BW}(t) + \alpha kr_{BW}(t) = -m \ddot{x}_{g}
$$

where $\ddot{x}_g$ describes the base acceleration input to the system, $x_{BW}$ describes the displacement of the system relative to the ground, and $r$ describes the non-physical Bouc-Wen hysteretic component. The term $\alpha \in \left[0,1\right]$ controls the influence of the linear and nonlinear displacement restoring forces on the system dynamics and is set to either $\alpha=0$ (linear) or $\alpha=1$ (fully nonlinear) for this example. 

Dividing out the mass allows for the statement of the dynamics in terms of the parameters $\xi$ and $\omega_n$. 

$$
\ddot{x}_{BW}(t) + 2 \xi \omega_n \dot{x}_{BW}(t) + \left(1-\alpha \right) \omega_n^2 x_{BW}(t) + \alpha \omega_n^2 r_{BW}(t) = -\ddot{x}_{g}
$$

The dynamics of the hysteretic component are further described by 

$$
\dot{r}_{BW}(t) = \dot{x}_{BW}(t) - \beta \left|\dot{x}_{BW}(t)\right| \left|r_{BW}(t)\right|^{n-1} r_{BW}(t)
-\gamma \dot{x}_{BW}(t) \left|r_{BW}(t)\right|^{n}.
$$

Collecting these dynamics in state space form results in

$$
x = \dot{\begin{bmatrix} x_{1} \\ x_{2} \\ x_{3} \end{bmatrix}} = 
\begin{bmatrix} 
x_{2} \\
-\ddot{x}_{g} \left(t\right) - 2 \xi \omega_n x_{2}(t) - \left(1-\alpha \right) \omega_n^2 x_{1}(t) - \alpha \omega_n^2 x_{3}(t) \\
x_{2}(t) - \beta \left|x_{2}(t)\right| \left|x_{3}(t)\right|^{n-1} x_{3}(t) - \gamma x_{2}(t) \left|x_{3}(t)\right|^{n}
\end{bmatrix}.
$$

The parameters of the system which will be inferred through this example can then be divided between the parameters for the linear system, $\xi = c/2m\omega_n=0.05$ and $\omega_n=\sqrt{k/m}=3$ rad/sec, and the parameters for the full Bouc-Wen system, $\beta=2$m<sup>-2</sup>, $n=2$, $\gamma=1$m<sup>-2</sup>. These parameters are collectively referred to as $\theta = \left[\xi, \omega_n, \beta, n, \gamma \right]$

### Transmission Probability Density Function
To work with the inference problem, this system of equations needs to be expressed in discrete form. The simplest way to do this in a manner which is uniform between inference algorithms is thrrough the Euler-Maruyama discretization scheme of stochastic ordinary differential equations<sup>2</sup>, as 

$$
x_{k+1} = \begin{bmatrix} x_{1} \\ x_{2} \\ x_{3} \end{bmatrix}_{k+1} = 
\begin{bmatrix} 
x_{1} + \Delta t x_{2} \\
x_{2} + \Delta t \left(-\ddot{x}_{g} - 2 \xi \omega_n x_{2} - \left(1-\alpha \right) \omega_n^2 x_{1} - \alpha \omega_n^2 x_{3} \right)\\
x_{3} + \Delta t \left(x_{2} - \beta \left|x_{2}\right| \left|x_{3}\right|^{n-1} x_{3} - \gamma x_{2} \left|x_{3}\right|^{n} \right)
\end{bmatrix}_{k}, 
$$

where the system is evaluated at a sampling frequency of $f_s=256$ Hz. To express the uncertainty inherent in this system, we assert the transmission density, we need to write the stochastic expressions for this dynamical system, as 

$$
p\left(x_{k+1}|x_{k}, \theta, \ddot{x}_{g,k}\right) = \mathcal{N} \left(x_{k+1} | x_{k}+ \Delta t \begin{bmatrix} 
x_{2} \\
-\ddot{x}_{g} - 2 \xi \omega_n x_{2} - \left(1-\alpha \right) \omega_n^2 x_{1} - \alpha \omega_n^2 x_{3} \\
x_{2} - \beta \left|x_{2}\right| \left|x_{3}\right|^{n-1} x_{3} - \gamma x_{2} \left|x_{3}\right|^{n} 
\end{bmatrix}_{k}, Q\right)
$$

where $Q$ represents the process noise associated with states. In this case, our process noise will not be inferred. Instead, it will be assumed known in the estimation of the states and parameters. The variance of this noise is modeled as 

$$
Q_{j,j} =  \left(\lambda \sqrt{\Delta t} \cdot \textrm{RMS}(x_{j}) \right)^2 \
$$

where $\lambda$ is a scaling factor to adjust the standard deviation of the process noise relative to the variation in the deterministic states and is set to $\eta = 2$% for this example. The factor $\sqrt{\Delta t}$ simulates the contribution of temporal discretization to the process noise. The true process is described by our stochastic model, indicating that the deterministic model cannot describe the full physics behind our observations. 

### Emission Probability Density Function
In this simulated experiment, we will assume that only acceleration information is collected, and that the collected data is contaminated with known, additive Gaussian noise. These observations of the absolute acceleration can therefore be modeled deterministically as

For the inference problem, we chose to observe the absolute acceleration of the system, given as 

$$
y_{k} = - 2 \xi \omega_n x_{2,k} - \left(1-\alpha \right) \omega_n^2 x_{1,k} - \alpha \omega_n^2 x_{3,k}
$$

and the uncertainty in these observations can be expressed as

$$
p\left(y_{k}|x_{k}, \theta\right) = \mathcal{N}\left(- 2 \xi \omega_n x_{2,k} - \left(1-\alpha \right) \omega_n^2 x_{1,k} - \alpha \omega_n^2 x_{3,k}, R \right), 
$$

where $R$ is the variance of the measurement process. This variance is modeled as

$$
R =  \left(\eta \cdot \textrm{RMS}(y)\right)^2,
$$

where $\bar{\eta}$ is a scaling factor to adjust the standard deviation of the process noise relative to the variation in the deterministic observations. In this example we will set $\eta = 20$%.

### Defining the Priors 
We are trying to infer the states $x_{1}, x_{2}$ and $x_{3}$ and the parameters $\xi, w_{n}, \beta, n,$ and $\gamma$. To generate the simulation data, we will use the values $\xi=0.05, w_{n}=3, \beta=2, n=2, \textrm{and} \gamma=1$, as discussed above. For inference, we will place the following priors on the states, assuming the system is at rest prior to excitation:

$$
p\left(x_{0}\right) = \mathcal{N} \left( \begin{bmatrix} 
0 \\
0 \\
0
\end{bmatrix}, 0.05^2\begin{bmatrix} 
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}\right)
$$

In this example, each inference algorithm is evaluated in multiple trials with different prior distributions on the parameters, representative of the prior distributions which might assumed in a typical experimental scenario. These prior distributions are given as
$$
p\left(\xi \right) = \mathcal{LogNormal} (\xi | \mu_{\xi}=\textrm{log}{\phi_{\xi}}, \sigma_{\xi}^2) \\
p\left(w_{n}\right) = \mathcal{LogNormal} (w_{n} | \mu_{w_{n}}=\textrm{log}{\phi_{w_{n}}}, \sigma_{w_{n}}^2)\\
p\left(\beta\right) = \mathcal{LogNormal} (\beta | \mu_{\beta}=\textrm{log}{\phi_{\beta}}, \sigma_{\beta}^2) \\
p\left(n\right) = \mathcal{LogNormal} (n | \mu_n=\textrm{log}{\phi_{n}}, \sigma_n^2)\\
p\left(\gamma\right) = \mathcal{LogNormal} (\gamma | \mu_{\gamma}=\textrm{log}{\phi_{\gamma}}, \sigma_{\gamma}^2),
$$

where the means of the distribution are Latin hypercube sampled in the ranges $\phi_\xi \in \left[0,1 \right]$,  $\phi_{\mu_{w_{n}}} \in \left[0,10 \right]$, $\phi_\beta \in \left[0,25 \right]$, $\phi_\gamma \in \left[0,25 \right]$, $\phi_n \in \left[0,5 \right]$.Variance levels are hand-selected to represent the uncertainty level a typical experimentalist might assign in association with the selected mean. The parameters of these prior distributions are recorded in the file **04-Data/parameter_priors.txt**, where the data is given in the following column layout over 50 inference trials: 

$$
\begin{matrix} \mu_{\xi} & | & \sigma_{\xi} & | & \mu_{w_{n}} & | & \sigma_{w_{n}} & | & \mu_n & | & \sigma_n & | & \mu_{\beta} & | & \sigma_{\beta} & | & \mu_{\gamma} & | & \sigma_{\gamma} \end{matrix}
$$

### Step 6: Base Excitation
The base excitation selected for this case study is a band-limited white noise (BLWN) signal with a maximum amplitude of 15 m/sec<sup>2</sup> and a cut-off frequency of 10 Hz, which excites the system for 10 sec. A secondary BLWN signal, also 10 sec long, is used to evaluate the predictive capacity of the inferred models, having a maximum amplitude of 10 m/sec<sup>2</sup> and a cut-off frequency of 20 Hz. The generation of these input signal and the corresponding responses of the true/noisy system are completed in the file **Generate_Input**.

__Developed by__: Alana Lund (Purdue University) \
__Last Updated__: 13 Sept. 2021 \
__License__: AGPL-3.0